In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import undetected_chromedriver
import pandas as pd
from time import sleep
from multiprocessing import Pool
from functools import partial
import os

In [2]:
def fetchMatch(matchID, driver):
    driver.get("https://csgostats.gg/match/" + str(matchID))
    
    mm = driver.find_element(By.XPATH,"//body/div[2]/div[1]/div[2]/div/div[1]/div[2]/div[1]/div")

    macthdate =  driver.find_elements( By.CLASS_NAME, 'match-date-text')
    if(macthdate.__len__()):
        macthdate = macthdate[0].get_attribute("textContent").strip()

    if(mm.get_attribute("textContent").strip()[:20] == "Official Matchmaking"):
        players =  driver.find_elements( By.CLASS_NAME, 'player-link')
        match = []
        for player in players:

            playerstats = {}
            playerstats["Match ID"] = matchID
            playerstats["Match Date"] = macthdate
            playerstats["Player ID"] = player.get_attribute('href')[-17:]
            print(playerstats["Player ID"])

            PCard = player.find_element(By.XPATH,"./..").find_element(By.XPATH,"./..")

            rank = PCard.find_elements(By.CLASS_NAME, "rank")
            if rank.__len__():
                playerstats["Rank"] = (rank[0].get_attribute("src"))[-6:-4].strip("/")

            rankupicon = PCard.find_elements(By.XPATH, "./td/div/div/span")

            if(rankupicon.__len__()):
                rankup = rankupicon[0].get_attribute("class")
                if rankup == "glyphicon glyphicon-chevron-up" :
                    playerstats["Rank Gain"] = 1

                elif rankup == "glyphicon glyphicon-chevron-down" :
                    playerstats["Rank Gain"] = -1

            else :
                playerstats["Rank Gain"] = 0        
            
            # print(stats["rank gain"])
            # teamScore = player.find_element(By.XPATH,"./..")
            # teamScore = teamScore.find_element(By.XPATH,"./..")
            team = PCard.find_element(By.XPATH,"./..")
            
            #teamScore = teamScore.find_element(By.ID,'match-scoreboard')
            #teamScore = teamScore.find_element(By.CLASS_NAME,'scoreboard')
            
            playerstats["Team"] = team.find_element(By.XPATH, "./tr/td/div").get_attribute("textContent").strip() [-1:]

            teams = team.find_element(By.XPATH,"./..")

            scores = teams.find_elements(By.XPATH, "./tbody/tr/td/div[2]")

            playerstats["Team 1 Score"] = scores[0].get_attribute("textContent").strip()
            playerstats["Team 2 Score"] = scores[1].get_attribute("textContent").strip()

            stats = PCard.find_elements(By.TAG_NAME, "td")

            # for stat in stats:
            #     print(stat.get_attribute("data-collapse"),stat.get_attribute("textContent").strip() )
            
            

            playerstats["K"] =                  stats[2].get_attribute("textContent").strip()
            playerstats["D"] =                  stats[3].get_attribute("textContent").strip()  
            playerstats["A"] =                  stats[4].get_attribute("textContent").strip()  
            playerstats["+/-"] =                stats[5].get_attribute("textContent").strip()  
            playerstats["K/D"] =                stats[6].get_attribute("textContent").strip()  
            playerstats["ADR"] =                stats[7].get_attribute("textContent").strip()  
            playerstats["HS%"] =                stats[8].get_attribute("textContent").strip()  
            playerstats["KAST"] =               stats[9].get_attribute("textContent").strip()  
            playerstats["Rating"] =             stats[10].get_attribute("textContent").strip()  
            playerstats["EF"] =                 stats[11].get_attribute("textContent").strip()  
            playerstats["FA"] =                 stats[12].get_attribute("textContent").strip()  
            playerstats["EBT"] =                stats[13].get_attribute("textContent").strip()  
            playerstats["UD"] =                 stats[14].get_attribute("textContent").strip()  
            playerstats["FKD"] =                stats[15].get_attribute("textContent").strip()  
            playerstats["FK"] =                 stats[16].get_attribute("textContent").strip() 
            playerstats["FD"] =                 stats[17].get_attribute("textContent").strip() 
            playerstats["FK as T"] =            stats[18].get_attribute("textContent").strip()  
            playerstats["FD as T"] =            stats[19].get_attribute("textContent").strip()  
            playerstats["FK as CT"] =           stats[20].get_attribute("textContent").strip()  
            playerstats["FD as CT"] =           stats[21].get_attribute("textContent").strip()  
            playerstats["Trade K"] =            stats[22].get_attribute("textContent").strip()  
            playerstats["Trade D"] =            stats[23].get_attribute("textContent").strip()  
            playerstats["Trade FK"] =           stats[24].get_attribute("textContent").strip()  
            playerstats["Trade FD"] =           stats[25].get_attribute("textContent").strip()  
            playerstats["Trade FK as T"] =      stats[26].get_attribute("textContent").strip()  
            playerstats["Trade FD as T"] =      stats[27].get_attribute("textContent").strip()  
            playerstats["Trade FK as CT"] =     stats[28].get_attribute("textContent").strip()  
            playerstats["Trade FD as CT"] =     stats[29].get_attribute("textContent").strip()  
            playerstats["1vX"] =                stats[30].get_attribute("textContent").strip()  
            playerstats["1v5"] =                stats[31].get_attribute("textContent").strip()  
            playerstats["1v4"] =                stats[32].get_attribute("textContent").strip()  
            playerstats["1v3"] =                stats[33].get_attribute("textContent").strip()  
            playerstats["1v2"] =                stats[34].get_attribute("textContent").strip()  
            playerstats["1v1"] =                stats[35].get_attribute("textContent").strip()  
            playerstats["3K+"] =                stats[36].get_attribute("textContent").strip()  
            playerstats["5K"] =                 stats[37].get_attribute("textContent").strip()  
            playerstats["4K"] =                 stats[38].get_attribute("textContent").strip()  
            playerstats["3K"] =                 stats[39].get_attribute("textContent").strip()  
            playerstats["2K"] =                 stats[40].get_attribute("textContent").strip()  
            playerstats["1K"] =                 stats[41].get_attribute("textContent").strip()  
        
            # print(playerstats["Team Score"])
            match.append(playerstats)
        return match
        
    else :
        return []


In [3]:
def multiFetch(offset):
    
    
    df = pd.DataFrame()

    # frame = []

    options = webdriver.ChromeOptions()
    # options.add_argument("--headless")
    driver = undetected_chromedriver.Chrome(options= options)

    # driver.implicitly_wait(0.1)
    # driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    for i in range(1):
        res = fetchMatch((offset + i),driver)
        # sleep(1)
        # print(res)
        df = pd.concat([pd.DataFrame(res),df])
        # driver.execute_script("window.open();")
        # driver.close()
        # driver.switch_to.window(driver.window_handles[-1])
        

    driver.quit()
    # print(frame)
    # df = pd.concat(frame)
    # display(df)
    return df

In [4]:
def testing(offset):
    print(offset)

In [5]:

        
df = multiFetch(78900001)

76561198072059812
76561198130530899
76561198400750497
76561198072974918
76561198405135921
76561198139108517
76561198318509550
76561198065331017
76561198969867185
76561198151331627


In [6]:

display(df)
df.to_csv("test.csv")

,Match ID,Match Date,Player ID,Rank,Rank Gain,Team,Team 1 Score,Team 2 Score,K,D,...,1v4,1v3,1v2,1v1,3K+,5K,4K,3K,2K,1K
0,78900001,13th Sep 2022 01:49:34,76561198072059812,11,0,1,12,16,27,21,...,0,0,0,0,3,0,0,3,3,12
1,78900001,13th Sep 2022 01:49:34,76561198130530899,10,0,1,12,16,23,23,...,0,0,0,0,2,0,0,2,2,13
2,78900001,13th Sep 2022 01:49:34,76561198400750497,11,0,1,12,16,17,23,...,0,0,0,1,0,0,0,0,5,7
3,78900001,13th Sep 2022 01:49:34,76561198072974918,11,0,1,12,16,13,20,...,0,0,0,0,0,0,0,0,1,11
4,78900001,13th Sep 2022 01:49:34,76561198405135921,10,0,1,12,16,13,22,...,0,0,0,0,0,0,0,0,3,7
5,78900001,13th Sep 2022 01:49:34,76561198139108517,12,0,2,12,16,29,17,...,0,0,0,0,1,0,0,1,8,10
6,78900001,13th Sep 2022 01:49:34,76561198318509550,10,0,2,12,16,23,16,...,0,0,0,0,1,0,0,1,5,10
7,78900001,13th Sep 2022 01:49:34,76561198065331017,11,0,2,12,16,21,21,...,0,0,0,1,0,0,0,0,7,7
8,78900001,13th Sep 2022 01:49:34,76561198969867185,11,0,2,12,16,17,17,...,0,0,0,0,0,0,0,0,5,7
9,78900001,13th Sep 2022 01:49:34,76561198151331627,11,1,2,12,16,18,23,...,0,0,0,0,1,0,1,0,3,8
